# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '9H9X9X4Q8QKNQ126',
  'HostId': 'clnxHoT7v9KrAGaLEMn/6EwMT7uzsxOYI9KJyh/v3C1g+qQwJCm4ItA6atnoD/Lj/M1s7AH0k8g=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'clnxHoT7v9KrAGaLEMn/6EwMT7uzsxOYI9KJyh/v3C1g+qQwJCm4ItA6atnoD/Lj/M1s7AH0k8g=',
   'x-amz-request-id': '9H9X9X4Q8QKNQ126',
   'date': 'Mon, 07 Apr 2025 16:04:26 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'cities-t

In [ ]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=8,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-04-07 11:04:28,361][INFO    ][coiled] Fetching latest package priorities...
[2025-04-07 11:04:28,362][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-04-07 11:04:29,294][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-04-07 11:04:29,302][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-04-07 11:04:30,323][INFO    ][coiled] Running pip check...
[2025-04-07 11:04:31,883][INFO    ][coiled] Validating environment...
[2025-04-07 11:04:34,198][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-04-07 11:04:34,437][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-04-07 11:04:34,438][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-04-07 11:04:36,207][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-utsov.dask.host/zdTzYHBektoVxYfZ/status


Task exception was never retrieved
future: <Task finished name='Task-1340' coro=<Client._gather.<locals>.wait() done, defined at /Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:2394> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py", line 2403, in wait
    raise AllExit()
distributed.client.AllExit
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/iostream.py", line 1367, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/ssl.py", line 1319, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)

During handling of the above exceptio

# RUN

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [7]:
import numpy as np
import pandas as pd
import geopandas as gpd
import math
import dask.dataframe as dd
import dask_geopandas as dgpd
from dask import delayed, compute, visualize
from citywide_calculation import get_utm_crs

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

def load_dataset(path, epsg=None):
    dataset = dgpd.read_parquet(path, npartitions=4)
    
    # Only assign if the file has no CRS
    if epsg:
        if dataset.crs is None:
            dataset = dataset.set_crs("EPSG:4326")  # assume WGS84 if missing
        dataset = dataset.to_crs(epsg)

    return dataset

In [8]:
from auxiliary_functions import *

# --- Step 1: Compute Bearings Vectorized ---


def metrics_roads_intersections(city_name):

    paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    roads = load_dataset(paths['roads'], epsg=epsg)
    intersections = load_dataset(paths['intersections'], epsg=epsg).compute()

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    intersections['osmid'] = intersections['osmid'].astype(int)
    intersection_angles = compute_intersection_angles(roads, intersections)
    street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
    intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
    intersection_angle_mapping = intersection_angle_mapping.compute()  

    intersections_with_angles_metric = intersections.merge(
        intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
    )

    joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
    average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()

    roads_with_tortuosity = calculate_tortuosity(roads.compute(), intersections)
    joined_tortuosity_grid = dgpd.sjoin(roads_with_tortuosity.set_geometry('road_geometry'), grid, predicate="within")
    average_tortuosity = joined_tortuosity_grid.groupby('index_right')['tortuosity'].mean()

    grid['metric_9'] = grid.index.map(average_tortuosity).astype(float)#.fillna(-999.).astype(float)
    grid['metric_10'] = grid.index.map(average_angle_between_roads).astype(float)#.fillna(-999.).astype(float)

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)

    return path
'''
# --- Example Usage ---

import time
start_time = time.time()  

cities = ['Nairobi']

tasks = [delayed(metrics_roads_intersections)(city) for city in cities]

results = compute(*tasks)

end_time = time.time()  
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")






#print(mapping.describe())'
'''


'\n# --- Example Usage ---\n\nimport time\nstart_time = time.time()  \n\ncities = [\'Nairobi\']\n\ntasks = [delayed(metrics_roads_intersections)(city) for city in cities]\n\nresults = compute(*tasks)\n\nend_time = time.time()  \nelapsed_time = end_time - start_time\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\n\n\n\n\n\n\n#print(mapping.describe())\'\n'

In [11]:
import dask
city_name = 'Nairobi'

paths = {
'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
}

epsg = get_epsg(city_name).compute()
grid = load_dataset(paths['grid'], epsg=epsg)
roads = load_dataset(paths['roads'], epsg=epsg)

included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']

def highway_filter(highway_value):
    # If highway_value is missing, return False
    if pd.isna(highway_value):
        return False
    # Split the string by commas, and strip any whitespace from each part
    types = [part.strip() for part in highway_value.split(',')]
    # Return True if any of the types is in our included list
    return any(t in included_road_types for t in types)

# Now filter the roads GeoDataFrame:
roads = roads[roads['highway'].apply(highway_filter)]

intersections = load_dataset(paths['intersections'], epsg=epsg).compute()
#intersections = intersections[intersections.street_count>2.0]

if 'geom' in grid.columns:
    grid = grid.drop(columns=['geom'])

intersections['osmid'] = intersections['osmid'].astype(int)
intersection_angles = compute_intersection_angles(roads, intersections)
street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()


intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
with dask.config.set({"optimization.fuse.active": False}):
    intersection_angle_mapping = intersection_angle_mapping.compute()

intersections_with_angles_metric = intersections.merge(
    intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
)

joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()

roads_with_tortuosity = calculate_tortuosity(roads.compute(), intersections)
joined_tortuosity_grid = dgpd.sjoin(roads_with_tortuosity.set_geometry('road_geometry'), grid, predicate="within")
average_tortuosity = joined_tortuosity_grid.groupby('index_right')['tortuosity'].mean()

grid['metric_9'] = grid.index.map(average_tortuosity).astype(float)#.fillna(-999.).astype(float)
#grid['metric_10'] = grid.index.map(average_angle_between_roads).astype(float)#.fillna(-999.).astype(float)

#path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'

if 'geom' in grid.columns:
    grid = grid.drop(columns='geom')


/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/dask/dataframe/dask_expr/_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('highway', 'bool'))

  warnings.warn(meta_warning(meta))


ValueError: Cannot fuse tasks with multiple outputs {('assign-b3d3d84184a87be673d402be7e774d9b', 0), ('centroid-07f8fffa36db281ba2e4a8f7801efcef', 0), ('getitem-0eb7f5133e31c530e17cb6ae37b86d73', 0), ('getitem-69c3de5586e15a4c8a5efd0826324726', 0), ('getitem-24fe38f4e5389a9c70e47a69b07c4107', 0), ('assign-0cdf25dcbc76f7afd32a3f2ec8199ce9', 0), ('centroid-b3179baec80c5b9f423832d7669184af', 0), ('operation-5c7492a5dfd9eea41b581387519ca569', 0), ('getitem-3cdccb78abf0273edc3d39a770962660', 0), ('degrees-90f3f8911cf9b5532606b0d14de91526', 0), ('getitem-5fd8c30938e2e04c8b719ee1910f6905', 0), ('x-8a2e490f66e46eb60811f0cb7da8ffb2', 0), ('y-bd04c43014643e34e96350df7cef06ad', 0), ('arctan2-8e5fb878a092c6c41412cdab7738da45', 0), ('sub-e8d6ca406ff0c471355128769144adb6', 0), ('mod-7e10d9c01dafa39e22e839d5ee8500f4', 0), ('sub-e655c7bdf00bc6173427fa21204a89cd', 0)}

In [10]:
city_name = 'Nairobi'
grid_cell_count = 0
paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
}
# Get EPSG
epsg = get_epsg(city_name).compute()
# Load grid
grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
if 'geom' in grid.columns:
    grid = grid.drop(columns=['geom'])

buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)#.compute()
buildings['distance_to_nearest_road'] = buildings['distance_to_nearest_road'].astype(float)
buildings['area'] = buildings.geometry.area
joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
counts_buildings = joined_buildings.groupby('index_right').size()
grid['n_buildings'] = grid.index.map(counts_buildings).fillna(0).astype(int)
average_distance = joined_buildings.groupby('index_right')['distance_to_nearest_road'].mean()
grid['average_distance_nearest_building'] = grid.index.map(average_distance).astype(float)#fillna(np.mean(average_distance)).astype(float)


/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/dask/dataframe/dask_expr/_collection.py:4729: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  warnings.warn(meta_warning(meta))
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/dask/dataframe/dask_expr/_collection.py:4729: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  warnings.warn(meta_warning(meta))


In [13]:
buildings_df = buildings.compute()

In [28]:
grid

,city_name,geometry,n_buildings,average_distance_nearest_building,n_buildings_closer_than_20m,m1_raw,m2_raw
npartitions=1,,,,,,,
,object,geometry,int64,float64,float64,float64,float64
,...,...,...,...,...,...,...


TypeError: 'LocIndexer' object does not support item assignment

In [15]:
buildings_df = buildings.compute()

In [31]:
buildings_df.loc['08b7a6e5420d4fff0200d63cb53248b8'] #.iloc['[08b7a6e551b4afff0200fa995601dbb9]']

geometry                    POLYGON ((254315.57391753283 9851799.706125613...
distance_to_nearest_road                                                500.0
area                                                                730.67503
Name: 08b7a6e5420d4fff0200d63cb53248b8, dtype: object

In [17]:
intersections

,osmid,y,x,street_count,highway,ref,junction,railway,geometry
0,29780021,-1.347361,36.903720,4,nan,nan,nan,nan,POINT (266748.234 9850975.451)
1,29780039,-1.344646,36.914422,3,nan,nan,nan,nan,POINT (267939.313 9851276.764)
2,29780051,-1.347919,36.909213,3,motorway_junction,23-D,nan,nan,POINT (267359.786 9850914.292)
3,30030168,-1.289755,36.798038,4,nan,nan,nan,nan,POINT (254977.775 9857337.106)
4,30030169,-1.290804,36.798552,3,nan,nan,nan,nan,POINT (255035.055 9857221.024)
...,...,...,...,...,...,...,...,...,...
134766,12669184297,-1.124174,36.827253,3,nan,nan,nan,nan,POINT (258215.557 9875654.86)
134767,12669184298,-1.126886,36.836207,3,nan,nan,nan,nan,POINT (259212.697 9875355.571)
134768,12669184337,-1.125271,36.824703,3,nan,nan,nan,nan,POINT (257931.723 9875533.353)
134769,12669184338,-1.125187,36.827387,3,nan,nan,nan,nan,POINT (258230.571 9875542.778)


In [13]:
grid_df = grid.compute()

/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 111.86 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [14]:
grid_df.to_parquet('metric_9_test_within.parquet')

In [25]:
intersections_with_angles_metric.to_parquet('intersections_with_angles.geoparquet')

In [ ]:
intersections_df['osmid'] = intersections_df['osmid'].astype('int')

def calculate_tortuosity(roads_df, intersections_df):

    # Merge roads with intersections for start point (u)
    roads_with_start = roads_df.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='u',
        right_on='osmid',
        how='left',
        suffixes=('', '_start')
    )

    # Merge with intersections again for end point (v)
    roads_with_both = roads_with_start.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='v',
        right_on='osmid',
        how='left',
        suffixes=('', '_end')
    )

    # Rename the merged geometry columns for clarity
    roads_with_both = roads_with_both.rename(
        columns={'geometry': 'road_geometry',
                'geometry_start': 'start_geom',
                'geometry_end': 'end_geom'}
    )

    # --- Step 2: Compute Straight-Line Distance between Intersections ---

    # Use the .distance method from GeoPandas (this assumes a projected CRS)
    roads_with_both['straight_distance'] = roads_with_both.apply(
        lambda row: row['start_geom'].distance(row['end_geom']), axis=1
    )

    # --- Step 3: Compute Road Length (if not already present) ---

    if 'length' not in roads_with_both.columns:
        roads_with_both['length'] = roads_with_both['road_geometry'].length

    # --- Step 4: Compute Tortuosity ---
    # Tortuosity is defined as road_length divided by the straight-line distance.
    # To avoid division by zero, we use np.where to set tortuosity to NaN when straight_distance is 0.

    roads_with_both['tortuosity'] = np.where(
        roads_with_both['straight_distance'] > 0,
        roads_with_both['straight_distance']/ roads_with_both['length'],
        np.nan
    )
    
    return roads_with_both

# --- Optionally: Check Summary Statistics ---
print(roads_with_both['tortuosity'].describe())

# The resulting roads_with_both GeoDataFrame now has a 'tortuosity' column.

count    320224.000000
mean          0.965383
std           0.092962
min           0.007647
25%           0.991078
50%           0.996044
75%           0.999557
max           1.001610
Name: tortuosity, dtype: float64


In [15]:
roads_with_both.columns

NameError: name 'roads_with_both' is not defined

In [26]:
client.close()